In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

from typing import Union, List, Tuple
from numpy.typing import ArrayLike, NDArray
from loguru import logger
from scipy.linalg import qr
from pyFDN.fdn import FDN
from pyFDN.feedback_matrix import FeedbackMatrixType
from pyFDN.utils import ms_to_samps, estimate_echo_density
from IPython.display import Image

#### The structure of the generalised Feedback Delay Network is shown below. In this notebook, we will play with the parameters of the FDN, and listen to its output.

In [ ]:
Image("../../resources/images/FDN_architecture.png")

\begin{aligned}
\boldsymbol{s}(n) & =\boldsymbol{A} \boldsymbol{s}(n-m)+\boldsymbol{b} x(n) \\
y(n) & =\boldsymbol{c}^T \boldsymbol{s}(n)+d x(n), \\
\boldsymbol{s}(n) & =\left[s_1(n), s_2(n), \ldots, s_N(n)\right]^T \\
\boldsymbol{s}(n-m) & =\left[s_1\left(n-m_1\right), s_2\left(n-m_2\right), \ldots, s_N\left(n-m_N\right)\right]^T
\end{aligned}

### Helper functions

In [ ]:
def generate_coprime_delay_line_lengths(delay_range_ms: ArrayLike, num_delay_lines: int,  fs: float) -> ArrayLike:
    delay_range_samps = ms_to_samps(delay_range_ms, fs)
    # generate prime numbers in specified range
    prime_nums = np.array(list(sp.primerange(delay_range_samps[0], delay_range_samps[1])), dtype=np.int32)
    rand_primes = prime_nums[np.random.permutation(len(prime_nums))]
    # delay line lengths
    delay_lengths = np.array(np.r_[rand_primes[:num_delay_lines - 1], sp.nextprime(delay_range_samps[1])], dtype=np.int32)
    return delay_lengths


#### Define FDN parameters

In [ ]:
# sampling rate
fs = 48000
# number of delay lines
N = 8
frame_size = 2**8

# we want a binaural output
num_input = 1
num_output = 2

# input gains
b = np.random.randn(N, num_input)
c = np.random.randn(num_output, N)
direct_gain = 0.5 * np.ones((num_output, num_input))

# delay lengths should be co-prime
# constrict delay range to be between 50 and 100ms
delay_range_ms = np.array([50, 100])
delay_lengths = generate_coprime_delay_line_lengths(delay_range_ms, N, fs)
logger.info(f'The delay line lengths are {delay_lengths} samples')

# how long should the impulse response be
ir_len = ms_to_samps(300, fs)
# create an impulse
input_data = np.zeros((num_input, ir_len))
input_data[:, 0] = 1.0

# desired broadband T60
des_t60_ms = 500

#### Plot FDN IR for a scalar matrix vs a feedback matrix

In [ ]:
fdn = FDN(fs, num_input, num_output, N, frame_size)
fdn.init_io_gains(b, c)
fdn.init_direct_gain(direct_gain)
fdn.init_delay_line_lengths(delay_lengths)
fdn.init_feedback_matrix()
fdn.init_absorption_gains(des_t60_ms)
fdn.init_delay_lines()

sfm_fdn_ir = fdn.process(input_data)
del fdn

In [ ]:
time_vector = np.arange(0, ir_len/fs, 1.0/fs)
time_constant = (des_t60_ms*1e-3) / np.log(1000) 
exp_envelope = np.exp(-time_vector / time_constant)

plt.figure()
plt.plot(time_vector, sfm_fdn_ir.T)
plt.plot(time_vector, np.stack((exp_envelope, -exp_envelope), axis=-1), 'k--')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'FDN IR with scalar feedback matrix and T60 of {des_t60_ms} ms')

In [ ]:
fdn = FDN(fs, num_input, num_output, N, frame_size)
fdn.init_io_gains(b, c)
fdn.init_direct_gain(direct_gain)
fdn.init_delay_line_lengths(delay_lengths)
fdn.init_feedback_matrix(FeedbackMatrixType.FILTER_VELVET, sparsity=0.2, num_mixing_stages=1)
fdn.init_absorption_gains(des_t60_ms)
fdn.init_delay_lines()

vfm_fdn_ir = fdn.process(input_data)

In [ ]:
fig, ax = plt.subplots(N, N, figsize=(12,12))
for i in range(N):
    for j in range(N):
        ax[i,j].stem(fdn.feedback.feedback_matrix[i, j, :])
        if j < N:
            ax[i,j].set_xlabel('')
        if i > 0:
            ax[i,j].set_ylabel('')
fig.suptitle("Velvet feedback matrix")
fig.tight_layout()

plt.figure()
plt.plot(time_vector, vfm_fdn_ir.T)
plt.plot(time_vector, np.stack((exp_envelope, -exp_envelope), axis=-1), 'k--')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'FDN IR with velvet feedback matrix and T60 of {des_t60_ms} ms')

del fdn

#### Compare the echo densities of the IRs with different feedback matrices

In [ ]:
ned_sfm = estimate_echo_density(sfm_fdn_ir.T, fs)
ned_vfm = estimate_echo_density(vfm_fdn_ir.T, fs)

plt.figure()
plt.plot(time_vector, ned_sfm, 'b')
plt.plot(time_vector, ned_vfm, 'r')
plt.xlabel('Time (s)')
plt.ylabel('NED')